In [1]:
from csv import excel

import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [3]:
time_step = []
temps = []

In [47]:
data=pd.read_csv('train.csv')
data.head()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,MYRUSD,SGDUSD,THBUSD,USDIDR,Harga
0,2022-01-01,Bawang Merah,Aceh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28970.0
1,2022-01-01,Bawang Merah,Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20870.0
2,2022-01-01,Bawang Merah,Banten,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26890.0
3,2022-01-01,Bawang Merah,Aceh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28970.0
4,2022-01-01,Bawang Merah,Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20870.0


In [48]:
df = data.drop(['Bahan_pangan', 'Daerah', 'Harga_lag_1', 'Harga', 'Harga_lag_7', 'Harga_lag_30', 'Google_trends'], axis=1)
df.head()

,Tanggal,MYRUSD,SGDUSD,THBUSD,USDIDR
0,2022-01-01,0.0,0.0,0.0,0.0
1,2022-01-01,0.0,0.0,0.0,0.0
2,2022-01-01,0.0,0.0,0.0,0.0
3,2022-01-01,0.0,0.0,0.0,0.0
4,2022-01-01,0.0,0.0,0.0,0.0


In [49]:
df.shape

(471049, 5)

In [50]:
df.duplicated().sum()

np.int64(470010)

In [51]:
df = df.drop_duplicates(ignore_index=True)

In [52]:
df.shape

(1039, 5)

In [53]:
df.head(10)

,Tanggal,MYRUSD,SGDUSD,THBUSD,USDIDR
0,2022-01-01,0.0,0.0,0.0,0.0
1,2022-01-02,0.0,0.0,0.0,0.0
2,2022-01-03,3404.7903,10543.6882,429.3135,14215.0
3,2022-01-04,3425.3235,10556.0478,429.6724,14283.5996
4,2022-01-05,3438.2169,10611.0453,432.7637,14385.5
5,2022-01-06,3446.2071,10640.3427,434.5073,14446.5
6,2022-01-07,3421.515,10586.8006,429.4102,14408.0
7,2022-01-08,3421.515,10586.8006,429.4102,14408.0
8,2022-01-09,3421.515,10586.8006,429.4102,14408.0
9,2022-01-10,3403.4944,10557.4189,426.3235,14318.5


In [68]:
df.drop(index=120, inplace=True)

In [ ]:
df.values[120]

array(['2022-04-30', '3325.0029', '10459.43', '421.1535', '14502.0'],
      dtype=object)

In [71]:
df.shape

(1038, 5)

In [74]:
for index, value in enumerate(df['USDIDR'].values):
    try:
        temps.append(round(float(value), 2))
        time_step.append(index)
    except Exception as e:
        print(f'At index {index}')
        print(e)

In [79]:
df.head

<bound method NDFrame.head of          Tanggal     MYRUSD      SGDUSD    THBUSD      USDIDR
0     2022-01-01        0.0         0.0       0.0         0.0
1     2022-01-02        0.0         0.0       0.0         0.0
2     2022-01-03  3404.7903  10543.6882  429.3135     14215.0
3     2022-01-04  3425.3235  10556.0478  429.6724  14283.5996
4     2022-01-05  3438.2169  10611.0453  432.7637     14385.5
...          ...        ...         ...       ...         ...
1034  2024-09-26  3682.0878  11786.1184  463.8846     15201.5
1035  2024-09-27  3640.0967  11737.8572   465.612     15070.0
1036  2024-09-28  3640.0967  11737.8572   465.612     15070.0
1037  2024-09-29  3640.0967  11737.8572   465.612     15070.0
1038  2024-09-30  3670.3084   11804.482  467.3261     15118.0

[1038 rows x 5 columns]>

In [76]:
series = np.array(temps)

In [77]:
# Normalization Function. DO NOT CHANGE THIS CODE
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)

# DO NOT CHANGE THIS CODE
split_time = 2500

time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

# DO NOT CHANGE THIS CODE
window_size = 64
batch_size = 256
shuffle_buffer_size = 1000

train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(train_set)
print(x_train.shape)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[None, 1]),
    tf.keras.layers.Dense(30, 'relu'),
    tf.keras.layers.Dense(20, 'relu'),
    tf.keras.layers.Dense(10, 'relu'),
    # YOUR CODE HERE.
    tf.keras.layers.Dense(1),
])

model.summary()

sgd = tf.keras.optimizers.SGD(momentum=0.9, learning_rate=2e-5)
huber = tf.keras.losses.Huber()

model.compile(
    loss=huber,
    optimizer=sgd,
    metrics=['mae']
)

class StopTrainWhenReach02(tf.keras.callbacks.Callback):
    def __init__(self, monitor, target):
        super(StopTrainWhenReach02, self).__init__()
        self.monitor = monitor
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is not None:
            if current < self.target:
                print(f'\nEpoch {epoch + 1}: {self.monitor} have reached below {self.target}. Stopping training.')
                self.model.stop_training = True

stop_training = StopTrainWhenReach02('mae', 0.13)

model.fit(
    train_set,
    epochs=200,
    callbacks=[stop_training]
)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(1038,)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, None, 30)       │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 20)       │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10)       │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, None, 1)        │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 901 (3.52 KB)

 Trainable params: 901 (3.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.3378 - mae: 0.8214
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3370 - mae: 0.8204  
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3370 - mae: 0.8204  
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3359 - mae: 0.8190  
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3346 - mae: 0.8175  
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.3333 - mae: 0.8159
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3318 - mae: 0.8141 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3307 - mae: 0.8127 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3285 - mae: 0.8100  
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.3277 - mae: 0.8089
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3263 - mae: 0.8072  
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3237 - mae: 0.8040 
Epoch 13/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4

In [ ]:
# model.save("model_B5.h5")